In [1]:
import pandas as pd
import numpy as np
import copy
from collections import deque
from pandas.core.frame import DataFrame

In [2]:

#Sample Data
df = pd.DataFrame(np.random.randint(0,1000,size=(10000, 1)),columns = ["time"])
df['GroupKey'] = np.random.choice(a=[10, 20, 30,40,50,60,70,80,90,100],size=10000,p=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
df['Cat1'] = np.random.choice(a=[100, 200, 300,400,99,33,45,6,7,8,3,3333,32],size=10000,p=[1/13,1/13,1/13,1/13,1/13,1/13,1/13,1/13,1/13,1/13,1/13,1/13,1/13])
df['ID'] = df.index


In [3]:
df

,time,GroupKey,Cat1,ID
0,362,60,33,0
1,655,70,8,1
2,360,100,100,2
3,865,50,400,3
4,447,20,3,4
...,...,...,...,...
9995,295,30,8,9995
9996,106,30,7,9996
9997,58,100,400,9997
9998,557,60,400,9998


In [4]:

#sort data
#Notice:You need to sort data by  yourself for the purpose of Rolling_Mode_Function can get correct mode feature(s)
df = df.sort_values(["GroupKey","time"])


In [5]:
df

,time,GroupKey,Cat1,ID
8173,1,10,45,8173
9908,1,10,33,9908
6484,2,10,8,6484
9854,2,10,100,9854
6244,3,10,3,6244
...,...,...,...,...
12,995,100,45,12
1280,996,100,8,1280
2526,996,100,3,2526
4333,998,100,8,4333


In [16]:

#Part1:Get Mode Function
def Get_Mode(mode_dic):
    MaxCount = 0;tmp_mode_deque = deque();
    for i in mode_dic.keys():
        if mode_dic[i]>=MaxCount:
            MaxCount=mode_dic[i]
            tmp_mode_deque.append([i,mode_dic[i]])
    mode_list=[]
    for i in range(len(tmp_mode_deque)):
        if tmp_mode_deque[i][1]>= MaxCount:
            mode_list.append(tmp_mode_deque[i][0])
    return mode_list

#Part2:Main Fucntion with Leakage option   
def Main_Mode_Function(df,GroupKey,JoinKey,Var,Leakage=False):
    tmp = copy.deepcopy(df)
    tmp_list = [list(tmp[JoinKey]),
                list(tmp[GroupKey]),
                list(tmp[Var].astype(str)),
                [0]*tmp.shape[0]
                ]
    data=[]
    for j in range(len(tmp_list[0])):
        data.append([tmp_list[i][j] for i in range(len(tmp_list))])

    GroupKey_bool = None;start_point=0
    mode_dic={}
    for i in range(len(data)):
        if data[i][1]!=GroupKey_bool:
            if (Leakage==True and i>0):
                save_point=i
                Leakage_Mode(data,mode_dic,start_point,save_point)
                start_point=save_point
            GroupKey_bool=data[i][1]
            print(data[i], mode_dic)
            mode_dic={}
            mode_dic[data[i][2]]=mode_dic.get(data[i][2],0)+1
            data[i][2] = [data[i][2]]
            data[i][3] = len([data[i][2]])
        else:
            mode_dic[data[i][2]]=mode_dic.get(data[i][2],0)+1
            mode_value = Get_Mode(mode_dic)
            data[i][2] = mode_value
            data[i][3] = len(mode_value)

    save_point=i+1
    Leakage_Mode(data,mode_dic,start_point,save_point)

    data=DataFrame(data)
    if Leakage==False:
        data.columns = [str(JoinKey),str(GroupKey),Var+"_mode",Var+"_mode_count"]
    else:
        data.columns = [str(JoinKey),str(GroupKey),Var+"_mode",Var+"_mode_count",Var+"_mode_Leakage"]
    globals()[Var + "_ModeFrame" ] = data
    print("Done: "+Var+"_ModeFrame")

#Part3:Grouping Leakage Mode
def Leakage_Mode(data,mode_dic,start_point,save_point):
    mode_dic_list = {value: key for key, value in mode_dic.items()}
    mode_dic_list = [(str(mode_dic_list[d])) for d in sorted(mode_dic_list.keys(),reverse = True)]
    for j in range(start_point,save_point,1):
        data[j].append([])
        for l in range(len(mode_dic_list)):
            if mode_dic_list[l] in set(data[j][2]):
               data[j][4].append(mode_dic_list[l])
               break;              

#Input four parameters:
#Rolling_Mode_Function(DataFrame,Grouping Key,JoinKey,Feature)
import time
tStart = time.time()

Main_Mode_Function(df,'GroupKey','ID','Cat1',Leakage=True)
tEnd = time.time()
print ("It cost %f sec" % (tEnd - tStart))

[8173, 10, '45', 0] {}
[1914, 20, '300', 0] {'45': 87, '33': 93, '8': 83, '100': 84, '3': 83, '32': 59, '99': 79, '200': 71, '7': 73, '400': 85, '3333': 71, '6': 83, '300': 71}
[784, 30, '8', 0] {'300': 69, '200': 74, '33': 77, '32': 63, '3333': 87, '6': 66, '400': 70, '45': 80, '8': 94, '99': 103, '3': 70, '7': 76, '100': 64}
[4631, 40, '100', 0] {'8': 67, '99': 99, '7': 66, '33': 81, '32': 85, '100': 81, '200': 83, '3': 79, '3333': 90, '45': 70, '400': 63, '6': 89, '300': 65}
[2651, 50, '33', 0] {'100': 69, '400': 67, '300': 78, '6': 73, '32': 66, '3': 88, '200': 81, '8': 63, '45': 65, '3333': 69, '99': 80, '7': 76, '33': 68}
[6195, 60, '200', 0] {'33': 62, '3': 83, '300': 88, '7': 76, '100': 73, '3333': 84, '32': 80, '400': 69, '8': 70, '6': 84, '45': 79, '200': 88, '99': 78}
[1259, 70, '100', 0] {'200': 82, '3333': 67, '33': 81, '3': 83, '300': 68, '7': 70, '45': 85, '400': 71, '6': 64, '100': 66, '99': 92, '8': 81, '32': 84}
[3120, 80, '99', 0] {'100': 89, '32': 67, '3333': 74, '3

In [8]:
Cat1_ModeFrame

,ID,GroupKey,Cat1_mode,Cat1_mode_count,Cat1_mode_Leakage
0,8173,10,[45],1,[45]
1,9908,10,"[45, 33]",2,[33]
2,6484,10,"[45, 33, 8]",3,[33]
3,9854,10,"[45, 33, 8, 100]",4,[33]
4,6244,10,"[45, 33, 8, 100, 3]",5,[33]
...,...,...,...,...,...
9995,12,100,[300],1,[300]
9996,1280,100,[300],1,[300]
9997,2526,100,[300],1,[300]
9998,4333,100,[300],1,[300]
